In [294]:
import tensorflow as tf
import numpy as np

In [295]:
tf.__version__

'1.4.0'

In [296]:
tf.reset_default_graph()
sess = tf.InteractiveSession()


In [297]:
PAD = 0
START = 1
EOS = 2

num_class = 7
hidden_unit = 7
num_layer = 3
batch_size = 3
max_step_decoder = 5
max_step_encoder = 4


encoder_input = [
    [5,6,3,4],
    [6,5,3,0],
    [5,6,0,0]
]

decoder_input = [
    [START,5,6,3,4],
    [START,6,5,3,0],
    [START,5,6,0,0]
]

target = [
    [5,6,3,4,EOS],
    [6,5,3,EOS,0],
    [5,6,EOS,0,0]
]

In [298]:
encoder_input

[[5, 6, 3, 4], [6, 5, 3, 0], [5, 6, 0, 0]]

In [299]:
def encoding(encoder_input_one_hot):
    # arg 1 [batchsize timestep ...]
    with tf.variable_scope('encoder'):
        cell = tf.contrib.rnn.LSTMCell(num_units=hidden_unit,state_is_tuple=True)
        initial_state = cell.zero_state(batch_size,tf.float32)
        enc_outputs, enc_states = tf.nn.dynamic_rnn(
            cell,
            encoder_input_one_hot,
            initial_state=initial_state,
            dtype=tf.float32)
        
    return enc_outputs, enc_states

In [300]:
def decoding(decoder_input_one_hot,enc_states):
    # arg 1: [batchsize timestep ...]
    # arg 2: 1D array list of hidden state
    pass

In [281]:
encoder_input = tf.constant(encoder_input,tf.int32)
decoder_input = tf.constant(decoder_input,tf.int32)

one_hot_encoder_input = tf.one_hot(encoder_input,depth=num_class,dtype=tf.float32)
one_hot_decoder_input = tf.one_hot(decoder_input,depth=num_class,dtype=tf.float32)

In [282]:
enc_outputs, enc_states = encoding(one_hot_encoder_input)

enc_outputs

<tf.Tensor 'encoder/rnn/transpose:0' shape=(3, 4, 7) dtype=float32>

In [283]:
enc_states

LSTMStateTuple(c=<tf.Tensor 'encoder/rnn/while/Exit_2:0' shape=(3, 7) dtype=float32>, h=<tf.Tensor 'encoder/rnn/while/Exit_3:0' shape=(3, 7) dtype=float32>)

In [284]:
decoder_input

<tf.Tensor 'Const_1:0' shape=(3, 5) dtype=int32>

In [285]:
# raw_rnn 은 time을 기준으로 계산 하기 때문에 [time batch-size ..]로 transpose로 inputs_ta를 넣어준다.

inputs_ta = tf.TensorArray(dtype=tf.float32, size=max_step_decoder)
inputs_ta = inputs_ta.unstack(tf.transpose(one_hot_decoder_input,[1,0,2]))

In [286]:


with tf.variable_scope('decoder'):
    # [batch_size time_step ..]
    # cell만들어 주기
    cell = tf.contrib.rnn.LSTMCell(num_units=hidden_unit,state_is_tuple=True)
    def loop_fn(time, cell_output, cell_state, loop_state):
        
        emit_output = cell_output  # == None for time == 0
        if cell_output is None:  # time == 0
            # time == 0 일때는 이전 cell_state를 넣어준다. 
            next_cell_state = enc_states
        else:
            next_cell_state = cell_state

        #time을 기준으로 모든 step들보다 time이 같거나 클경우까지 계속 진행된다.
        elements_finished = (time >= max_step_decoder)
        
        #모두 time을 기준으로 전부 true인지를 체크
        finished = tf.reduce_all(elements_finished)
        
        #끝나면 tf.zeros([batch_size, num_class]) 형태로 return 해줘야 한다.
        next_input = tf.cond(
          finished,
          lambda: tf.zeros([batch_size, num_class], dtype=tf.float32),
          lambda: inputs_ta.read(time))
        next_loop_state = None
    
        return (elements_finished, next_input, next_cell_state,
              emit_output, next_loop_state)

outputs_ta, final_state, _ = tf.nn.raw_rnn(cell, loop_fn)
outputs = outputs_ta.stack()

In [287]:
outputs = tf.transpose(outputs,[1,0,2])

In [288]:
decoder_logits = tf.contrib.layers.fully_connected(outputs, num_class,activation_fn=None)

In [289]:
decoder_logits

<tf.Tensor 'fully_connected/BiasAdd:0' shape=(3, ?, 7) dtype=float32>

In [290]:
decoder_prediction = tf.argmax(decoder_logits, 2)

In [291]:
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
            labels=tf.one_hot(target,depth=num_class,dtype=tf.float32),
            logits=decoder_logits,
        )
loss = tf.reduce_mean(stepwise_cross_entropy,name="loss")

In [292]:
train_op = tf.contrib.layers.optimize_loss(
            loss, tf.train.get_global_step(),
            optimizer='Adam',
            summaries=['loss', 'learning_rate'],
            learning_rate=0.1,
            name="train_op")

In [293]:
sess.run(tf.global_variables_initializer())

for i in range(0,2000):
    l, _ = sess.run([loss,train_op])
    print(l)
    if i%50 == 0:
        print(sess.run(decoder_prediction))


1.94495
[[6 6 6 6 2]
 [6 6 6 6 6]
 [6 6 6 6 6]]
1.78884
1.6598
1.51548
1.33092
1.12857
0.949453
0.801662
0.689052
0.623379
0.541927
0.481379
0.441851
0.393757
0.357727
0.329585
0.281796
0.263915
0.219761
0.197027
0.171442
0.150997
0.120006
0.10493
0.0865449
0.0681678
0.0560622
0.0433192
0.0345339
0.0280913
0.0234024
0.0200378
0.0175142
0.0154301
0.013604
0.0120595
0.0108035
0.00977983
0.00892624
0.00817489
0.00747601
0.00683751
0.00629519
0.0058598
0.00550483
0.00518782
0.00487853
0.00457478
0.00429622
0.00406324
0.00387756
[[5 6 3 4 2]
 [6 5 3 2 0]
 [5 6 2 0 0]]
0.00371699
0.00355334
0.0033832
0.00322776
0.00310195
0.00300121
0.00291137
0.00282134
0.00272968
0.00264305
0.00256892
0.00250853
0.0024547
0.00239835
0.00233877
0.00228256
0.00223438
0.00219266
0.00215245
0.00211068
0.0020684
0.00202892
0.00199419
0.00196283
0.00193174
0.00189953
0.00186792
0.00183879
0.00181242
0.00178738
0.00176232
0.00173714
0.00171296
0.00169056
0.00166961
0.00164927
0.00162893
0.00160877
0.00158946
0.00

0.000116617
0.000116355
0.000116101
0.000115886
0.00011564
0.000115394
0.000115179
0.000114949
0.00011471
0.000114496
0.000114273
0.000114019
0.000113773
0.000113574
0.000113336
0.000113105
[[5 6 3 4 2]
 [6 5 3 2 0]
 [5 6 2 0 0]]
0.000112859
0.000112636
0.000112446
0.000112215
0.000111977
0.000111738
0.0001115
0.000111309
0.000111071
0.000110848
0.000110642
0.000110411
0.000110197
0.000110006
0.000109776
0.000109569
0.000109347
0.000109116
0.00010891
0.000108679
0.000108465
0.00010825
0.000108051
0.000107845
0.000107598
0.000107424
0.000107201
0.000106971
0.00010678
0.000106589
0.000106383
0.000106176
0.000105954
0.000105747
0.000105548
0.000105358
0.000105135
0.000104929
0.000104722
0.000104515
0.000104333
0.00010411
0.000103927
0.000103721
0.000103514
0.000103347
0.000103125
0.000102934
0.000102743
0.000102553
[[5 6 3 4 2]
 [6 5 3 2 0]
 [5 6 2 0 0]]
0.000102354
0.000102163
0.000101965
0.000101782
0.000101583
0.000101377
0.000101186
0.000100995
0.00010082
0.000100622
0.000100423
0.000

4.08003e-05
4.07208e-05
4.06572e-05
4.06334e-05
4.05698e-05
4.05221e-05
4.04744e-05
4.04109e-05
4.03791e-05
4.03234e-05
4.02599e-05
4.02201e-05
4.01963e-05
4.01327e-05
4.0093e-05
4.00453e-05
3.99817e-05
3.9942e-05
3.98705e-05
3.98307e-05
3.97989e-05
3.97592e-05
3.96877e-05
3.96718e-05
3.964e-05
3.95685e-05
3.95208e-05
3.94652e-05
3.94175e-05
3.93857e-05
3.93301e-05
3.93062e-05
[[5 6 3 4 2]
 [6 5 3 2 0]
 [5 6 2 0 0]]
3.92506e-05
3.9195e-05
3.91394e-05
3.91235e-05
3.90837e-05
3.90202e-05
3.89407e-05
3.89168e-05
3.88612e-05
3.88294e-05
3.87738e-05
3.87182e-05
3.86625e-05
3.86308e-05
3.85751e-05
3.85433e-05
3.85036e-05
3.8448e-05
3.84321e-05
3.83606e-05
3.8297e-05
3.8289e-05
3.82573e-05
3.81937e-05
3.81381e-05
3.81063e-05
3.80824e-05
3.80427e-05
3.79712e-05
3.79155e-05
3.78838e-05
3.78599e-05
3.78122e-05
3.77487e-05
3.77089e-05
3.7701e-05
3.76374e-05
3.75818e-05
3.755e-05
3.74785e-05
3.74387e-05
3.74149e-05
3.73593e-05
3.73116e-05
3.72639e-05
3.72321e-05
3.71924e-05
3.71447e-05
3.71209e-05